In [ ]:
import os
import json

from IPython.display import display, Markdown

import numpy as np
import matplotlib.pyplot as plt

from pymedphys.level1.msqconnect import mosaiq_connect
from pymedphys.level1.configutilities import (
    get_cache_filepaths, get_mu_density_parameters,
    get_index, get_centre, get_sql_servers, get_sql_servers_list,
    get_filepath
)
from pymedphys.level1.mudensity import find_relevant_control_points, calc_mu_density
from pymedphys.level2.msqdelivery import multi_fetch_and_verify_mosaiq
from pymedphys.level3.analyselogfiles import (
    analyse_single_hash, find_consecutive_logfiles, get_field_id_key_map, plot_results,
    calc_comparison
)

from decode_trf.decode_trf import delivery_data_from_logfile

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
index = get_index(config)
file_hashes = np.array(list(index.keys()))

field_types = np.array([
    index[file_hash]['delivery_details']['field_type']
    for file_hash in file_hashes
])

file_hashes = file_hashes[field_types == 'DMLC']

is_qa = np.array([
    index[file_hash]['delivery_details']['qa_mode']
    for file_hash in file_hashes
])

file_hashes = file_hashes[np.invert(is_qa)]

machine = np.array([
    index[file_hash]['logfile_header']['machine']
    for file_hash in file_hashes
])

# limit to RCCC for now
file_hashes = file_hashes[(machine == '2619') | (machine == '2694')]
np.random.shuffle(file_hashes)

# might need course information
index[file_hashes[0]]

In [ ]:
patient_grouped_fields = dict()
field_id_grouped_hashes = dict()

for file_hash in file_hashes:
    delivery_details = index[file_hash]['delivery_details']
    patient_id = delivery_details['patient_id']
    field_id = delivery_details['field_id']
    
    if patient_id not in patient_grouped_fields:
        patient_grouped_fields[patient_id] = set()
        
    patient_grouped_fields[patient_id].add(field_id)
    
    if field_id not in field_id_grouped_hashes:
        field_id_grouped_hashes[field_id] = []
        
    field_id_grouped_hashes[field_id].append(file_hash)

In [ ]:
patient_ids = list(patient_grouped_fields.keys())

In [ ]:
patient_id = '013340'
fields = patient_grouped_fields[patient_id]

In [ ]:
patient_id

In [ ]:
for field in fields:
    print(field_id_grouped_hashes[field])

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    for field in fields:
        mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(
            cursor, field)
        print(set(mosaiq_delivery_data.gantry))

In [ ]:
def group_consecutive_logfiles(file_hashes, index):
    times = np.array([
        index[key]['local_time']
        for key in file_hashes
    ]).astype(np.datetime64)

    sort_reference = np.argsort(times)
    file_hashes = file_hashes[sort_reference]
    times = times[sort_reference]

    hours_4 = np.array(60 * 60 * 4).astype(np.timedelta64)
    split_locations = np.where(np.diff(times) >= hours_4)[0] + 1

    return np.split(file_hashes, split_locations)

######
# TODO Need to expand this out to run over all fields
######
field_id = list(fields)[0]
keys = np.array(field_id_grouped_hashes[field_id])

logfile_groups = group_consecutive_logfiles(keys, index)
logfile_groups = [
    tuple(group)
    for group in logfile_groups
]
logfile_groups

In [ ]:
field_id

In [ ]:
def assert_array_agreement(unique_logfile_gantry_angles, mosaiq_gantry_angles, allowed_deviation):
    difference_matrix = np.abs(unique_logfile_gantry_angles[:,None] - mosaiq_gantry_angles[None,:])
    agreement_matrix = difference_matrix <= allowed_deviation
    row_agreement = np.any(agreement_matrix, axis=1)
    all_rows_have_at_least_one_agreement = np.all(row_agreement)
    
    assert all_rows_have_at_least_one_agreement, (
        'There is a logfile gantry angle that deviates by more than {} degrees '
        'from the Mosaiq control points. Unsure how to handle this.\n\n'
        'Logfile: {}\nMosaiq: {}\nDifference Matrix:\n{}\n'
        'Agreement Matrix:\n{}'.format(
            gantry_tolerance, unique_logfile_gantry_angles, mosaiq_gantry_angles,
            difference_matrix, agreement_matrix))


In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(
        cursor, field_id)

In [ ]:
mosaiq_gantry_angles = np.unique(mosaiq_delivery_data.gantry)
mosaiq_gantry_angles

In [ ]:
def get_gantry_tolerance(index, file_hash, config):
    machine_name = index[file_hash]['logfile_header']['machine']
    machine_type = config['machine_map'][machine_name]['type']
    gantry_tolerance = config['machine_types'][machine_type]['gantry_tolerance']
    
    return gantry_tolerance

In [ ]:
def get_delivery_data_grouped_by_gantry_angle(index, config, logfile_groups):
    delivery_data_grouped_by_gantry_angle = dict()

    for logfile_group in logfile_groups:
        delivery_data_grouped_by_gantry_angle[logfile_group] = dict()

        for file_hash in logfile_group:
            filepath = get_filepath(index, config, file_hash)
            logfile_delivery_data = delivery_data_from_logfile(filepath)
            mu = np.array(logfile_delivery_data.monitor_units)

            relevant_control_points = find_relevant_control_points(mu)

            mu = mu[relevant_control_points]
            mlc = np.array(logfile_delivery_data.mlc)[relevant_control_points]
            jaw = np.array(logfile_delivery_data.jaw)[relevant_control_points]
            logfile_gantry_angles = np.array(logfile_delivery_data.gantry)[relevant_control_points]

            gantry_tolerance = get_gantry_tolerance(index, file_hash, config)
            unique_logfile_gantry_angles = np.unique(logfile_gantry_angles)

            assert_array_agreement(unique_logfile_gantry_angles, mosaiq_gantry_angles, gantry_tolerance)

            delivery_data_grouped_by_gantry_angle[logfile_group][file_hash] = dict()

            for mosaiq_gantry_angle in mosaiq_gantry_angles:
                delivery_data_grouped_by_gantry_angle[logfile_group][file_hash][mosaiq_gantry_angle] = dict()
                agrees_within_tolerance = (
                    np.abs(logfile_gantry_angles - mosaiq_gantry_angle) <= gantry_tolerance)

                delivery_data_grouped_by_gantry_angle[logfile_group][file_hash][mosaiq_gantry_angle]['mu'] = mu[agrees_within_tolerance]
                delivery_data_grouped_by_gantry_angle[logfile_group][file_hash][mosaiq_gantry_angle]['mlc'] = mlc[agrees_within_tolerance]
                delivery_data_grouped_by_gantry_angle[logfile_group][file_hash][mosaiq_gantry_angle]['jaw'] = jaw[agrees_within_tolerance]
                
    return delivery_data_grouped_by_gantry_angle


delivery_data_grouped_by_gantry_angle = get_delivery_data_grouped_by_gantry_angle(index, config, logfile_groups)

In [ ]:
def get_logfile_mu_density_by_gantry_angle(logfile_groups, mosaiq_gantry_angles, delivery_data_grouped_by_gantry_angle):
    logfile_mu_density_by_gantry_angle = dict()

    for logfile_group in logfile_groups:
        delivery_data = delivery_data_grouped_by_gantry_angle[logfile_group]
        logfile_mu_density_by_gantry_angle[logfile_group] = dict()

        for file_hash in logfile_group:
            for mosaiq_gantry_angle in mosaiq_gantry_angles:
                if len(delivery_data[file_hash][mosaiq_gantry_angle]['mu']) > 0:
                    mu_density = calc_mu_density(**delivery_data[file_hash][mosaiq_gantry_angle])
                    if mosaiq_gantry_angle not in logfile_mu_density_by_gantry_angle[logfile_group]:
                        logfile_mu_density_by_gantry_angle[logfile_group][mosaiq_gantry_angle] = list(mu_density)
                    else:
                        assert np.all(logfile_mu_density_by_gantry_angle[logfile_group][mosaiq_gantry_angle][0] == mu_density[0])
                        assert np.all(logfile_mu_density_by_gantry_angle[logfile_group][mosaiq_gantry_angle][1] == mu_density[1])
                        logfile_mu_density_by_gantry_angle[logfile_group][mosaiq_gantry_angle][2] += mu_density[2]
                        
    return logfile_mu_density_by_gantry_angle


logfile_mu_density_by_gantry_angle = get_logfile_mu_density_by_gantry_angle(logfile_groups, mosaiq_gantry_angles, delivery_data_grouped_by_gantry_angle)

In [ ]:
def get_mosaiq_delivery_data_by_gantry_angle(mosaiq_delivery_data):
    mu = np.array(mosaiq_delivery_data.monitor_units)
    mlc = np.array(mosaiq_delivery_data.mlc)
    jaw = np.array(mosaiq_delivery_data.jaw)
    gantry_angles = np.array(mosaiq_delivery_data.gantry)
    unique_mosaiq_gantry_angles = np.unicode(gantry_angles)

    mosaiq_delivery_data_by_gantry_angle = dict()

    for mosaiq_gantry_angle in unique_mosaiq_gantry_angles:
        gantry_angle_matches = gantry_angles == mosaiq_gantry_angle

        diff_mu = np.concatenate([[0], np.diff(mu)])[gantry_angle_matches]
        gantry_angle_specific_mu = np.cumsum(diff_mu)

        mosaiq_delivery_data_by_gantry_angle[mosaiq_gantry_angle] = dict()
        mosaiq_delivery_data_by_gantry_angle[mosaiq_gantry_angle]['mu'] = gantry_angle_specific_mu
        mosaiq_delivery_data_by_gantry_angle[mosaiq_gantry_angle]['mlc'] = mlc[gantry_angle_matches]
        mosaiq_delivery_data_by_gantry_angle[mosaiq_gantry_angle]['jaw'] = jaw[gantry_angle_matches]
        
mosaiq_delivery_data_by_gantry_angle = get_mosaiq_delivery_data_by_gantry_angle(mosaiq_delivery_data)

In [ ]:
def get_mosaiq_mu_density_by_gantry_angle(mosaiq_delivery_data_by_gantry_angle):
    mosaiq_mu_density_by_gantry_angle = dict()
    mosaiq_gantry_angles = mosaiq_delivery_data_by_gantry_angle.keys()

    for mosaiq_gantry_angle in mosaiq_gantry_angles:
        mu_density = calc_mu_density(**mosaiq_delivery_data_by_gantry_angle[mosaiq_gantry_angle])
        mosaiq_mu_density_by_gantry_angle[mosaiq_gantry_angle] = mu_density
        
    return mosaiq_mu_density_by_gantry_angle

In [ ]:
def comparison_results(mosaiq_mu_density_by_gantry_angle, logfile_mu_density_by_gantry_angle):
    comparison_results = dict()
    mosaiq_gantry_angles = mosaiq_mu_density_by_gantry_angle.keys()

    for mosaiq_gantry_angle in mosaiq_gantry_angles:
        comparison_results[mosaiq_gantry_angle] = dict()
        comparison_results[mosaiq_gantry_angle]['comparisons'] = []

        grid_xx = mosaiq_mu_density_by_gantry_angle[mosaiq_gantry_angle][0]
        grid_yy = mosaiq_mu_density_by_gantry_angle[mosaiq_gantry_angle][1]
        mosaiq_mu_density = mosaiq_mu_density_by_gantry_angle[mosaiq_gantry_angle][2]

        for logfile_group in logfile_groups:
            assert np.all(grid_xx == logfile_mu_density_by_gantry_angle[logfile_group][mosaiq_gantry_angle][0])
            assert np.all(grid_yy == logfile_mu_density_by_gantry_angle[logfile_group][mosaiq_gantry_angle][1])

            logfile_mu_density = logfile_mu_density_by_gantry_angle[logfile_group][mosaiq_gantry_angle][2]

            comparison = calc_comparison(logfile_mu_density, mosaiq_mu_density)
            comparison_results[mosaiq_gantry_angle]['comparisons'].append(comparison)
            
        comparison_results[mosaiq_gantry_angle]['median'] = np.median(comparison_results[mosaiq_gantry_angle]['comparisons'])
        ref = np.argmin(np.abs(
            comparison_results[mosaiq_gantry_angle]['comparisons'] - 
            comparison_results[mosaiq_gantry_angle]['median']
        ))
        comparison_results[mosaiq_gantry_angle]['filehashes'] = logfile_groups[ref]
        
    return comparison_results

comparison_results = comparison_results(mosaiq_mu_density_by_gantry_angle, logfile_mu_density_by_gantry_angle)
comparison_results